In [1]:
import spacy
from spacy.language import Language
import os
import docx2txt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
nlp = spacy.load('en_core_web_lg',disable = ['ner', 'parser'])
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print(nlp.pipe_names)

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer']


In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fishrak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fishrak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def all_file_paths(master_directory):   
    business_txt_list = []
    path_l = []
    for root, dirs, files in os.walk(master_directory):
        
        

        if files:
            for i in files:
                path = "{}\{}".format(root,i)
                if path == 'bbc_doc\desktop.ini':
                    continue
                path_l.append(path)
                
    
    return(path_l)

all_path = all_file_paths('bbc_doc')

for i in all_path:
    print(i)

bbc_doc\business\001.docx
bbc_doc\business\002.docx
bbc_doc\business\003.docx
bbc_doc\business\004.docx
bbc_doc\business\005.docx
bbc_doc\entertainment\001.docx
bbc_doc\entertainment\002.docx
bbc_doc\entertainment\003.docx
bbc_doc\entertainment\004.docx
bbc_doc\entertainment\005.docx
bbc_doc\politics\001.docx
bbc_doc\politics\002.docx
bbc_doc\politics\003.docx
bbc_doc\politics\004.docx
bbc_doc\politics\005.docx
bbc_doc\sports\001.docx
bbc_doc\sports\002.docx
bbc_doc\sports\003.docx
bbc_doc\sports\004.docx
bbc_doc\sports\005.docx
bbc_doc\technology\001.docx
bbc_doc\technology\002.docx
bbc_doc\technology\003.docx
bbc_doc\technology\004.docx
bbc_doc\technology\005.docx


In [5]:

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', '', text.lower())
    return text


def findline(word,doc_list): #Added doc_list as var
    line_num = []
    for i in range(len(doc_list)):
        if word in doc_list[i]:
            line_num.append(i+1)

    return line_num



In [6]:
def sim_word(path,query,score=0.6):
 
#FI
    text = docx2txt.process(path)


    tokens = nlp(text)
    lemma_list = []
    for token in tokens:
        if token.is_stop is False:
            token_preprocessed = preprocessor(token.text)
            if token_preprocessed != '':
                 lemma_list.append(nlp(token_preprocessed))
   

    tokens_str = text.lower()
    tokens_str = word_tokenize(tokens_str)
    tokens_str = [i for i in tokens_str if not i in stop_words]    
#FI    
    
    query = query
    
    
#PN    
    wordfreq = []
    for w in tokens_str:
        wordfreq.append(tokens_str.count(w))


    for i in list(zip(tokens_str, wordfreq)):
        if query == i[0]:
            word_occurences = (i[0], i[1])
            break

    line = 0
    word_not_found = []

    for word in text:
        if word == '\n':
            line += 1

    doc_list = text.lower().splitlines()
    doc_list = [i for i in doc_list if i]

#PN


#FI & PN
    key = nlp(query)
    
    sim_score = []
    words = []

    pg_list = []

    for i in lemma_list:
        s = key.similarity(i)

        if s > score:
            words.append(i)
            sim_score.append(s)

            word_found_list = []
            word_found_list.append(str(i.text))
            word_found_lines = findline(word_found_list[0],doc_list)


            pg_list.append(tuple(word_found_lines))
            
    matrix = pd.DataFrame({'Similarity Score': sim_score,'paragraph': pg_list}, index = words)
    matrix = matrix.drop_duplicates()

    matrix =  matrix.sort_values(by=['Similarity Score'],ascending=False)

    print("For document: {}, path: {}, keyword: {}, Similarity Score Threshold: {}".format(os.path.basename(path),path,query,score))
    print(matrix)
    print('\n')

#FI & PN

In [13]:
for i in all_path[:]:
    sim_word(i,"international market",0.6)



For document: 001.docx, path: bbc_doc\business\001.docx, keyword: international market, Similarity Score Threshold: 0.5
             Similarity Score        paragraph
(market)             0.833610             (6,)
(business)           0.640088             (4,)
(financial)          0.616982             (5,)
(sales)              0.596859           (1, 3)
(company)            0.575999           (4, 6)
(investors)          0.566891             (3,)
(growth)             0.535442             (5,)
(exchange)           0.530614             (4,)
(offering)           0.527257             (4,)
(biggest)            0.525046             (3,)
(profit)             0.520769  (1, 2, 3, 4, 5)
(customers)          0.520677             (4,)
(profits)            0.515995     (2, 3, 4, 5)


For document: 002.docx, path: bbc_doc\business\002.docx, keyword: international market, Similarity Score Threshold: 0.5
               Similarity Score paragraph
(market)               0.833610      (3,)
(trade)         

For document: 005.docx, path: bbc_doc\sports\005.docx, keyword: international market, Similarity Score Threshold: 0.5
         Similarity Score  paragraph
(world)          0.624327  (1, 2, 3)


For document: 001.docx, path: bbc_doc\technology\001.docx, keyword: international market, Similarity Score Threshold: 0.5
                 Similarity Score     paragraph
(international)          0.843367         (10,)
(world)                  0.624327          (8,)
(countries)              0.615304          (8,)
(country)                0.592343        (2, 9)
(sectors)                0.553917          (3,)
(profit)                 0.520769         (10,)
(government)             0.508514  (3, 4, 6, 8)
(technology)             0.506727        (3, 5)
(products)               0.502810          (9,)


For document: 002.docx, path: bbc_doc\technology\002.docx, keyword: international market, Similarity Score Threshold: 0.5
              Similarity Score  paragraph
(world)               0.624327       (

### Trying with transformer models 

In [8]:
# from sentence_transformers import SentenceTransformer, util
# import numpy as np

# model = SentenceTransformer('stsb-roberta-large')

# sentence1 = "Word"
# sentence2 = "Word"
# # encode sentences to get their embeddings
# embedding1 = model.encode(sentence1, convert_to_tensor=True)
# embedding2 = model.encode(sentence2, convert_to_tensor=True)
# # compute similarity scores of two embeddings
# cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
# print("Sentence 1:", sentence1)
# print("Sentence 2:", sentence2)
# print("Similarity score:", cosine_scores.item())


# key = "learn"

# embedding1 = model.encode(key, convert_to_tensor=True)

# for i in lemma_list:
#     i = i.text
     
#     embedding2 = model.encode(i, convert_to_tensor=True)
#     # compute similarity scores of two embeddings
#     cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    
#     if cosine_scores > 0.5:
#         print("Key:{} \nWord Found:{} \nSimilarity score:{:0.2f}".format(key,i,np.array(cosine_scores)[0][0]))
#         print("----------------------------------------------------")  